### 本文的主要贡献
提供一个新的网络架构：纯attention(包括self attention 和 multi head attention)架构。<br>
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms to draw global dependencies between input and output, dispensing with recurrentce and convolutions entirely. The Transformer allows for significantly more parallelization and can be reach a new state of the art in translation quality.<br>
self attention: sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.
- RNN的限制： 

RNN网络的输出$h_t$同时受$h_{t-1}$和$input_t$的影响。This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples.<br>
这里的前后关系不是很明白，可以理解，他阻碍了并行，但为什么说这点在长句中是致命的？？

- CNN的限制

同样存在RNN的困扰，不过情节比较轻而已。
[22, 17, 9]all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet.
- attention的优势

可以很好的模拟长句内，不同部分（单词，短语）的依赖关系。<br>
The number of operations required to realte signals from two arbitrary input or output positions is reduced to a constant, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions.(multi-head attention)<br>
allowing modeling of dependencies without regard to their distance in the input or output sequences.<br>

### 模型架构


### ？？？
RNN的限制中的as memory constraints limit batching across example不是很理解？？？
Model Architecture中说的competitive neural sequence transduction models是指什么？？为什么这么称呼？？

### 还要看的
[4] : reading comprehension, self attention<br>
[9] : ConvS2S<br>
[17]: ByteNet<br>
[20]: fractorization tricks<br>
[21]: task-independent sentence representations, self attention<br>
[22]: extended Neural GPU<br>
[26]: abstractive summarization, self attention<br>
[27]: textual entailment, self attention<br>
[31]: conditional computation